In [92]:
# Importando biblioteca
import requests                         # Para fazer requisições HTTP
import json                             # Para manipular JSON
import os                               # Para manipular variáveis de ambiente
from dotenv import load_dotenv          # Para carregar variáveis de ambiente do arquivo .env
import pandas as pd                     # Para manipular DataFrames

# Lendo a chave da API do arquivo .env
load_dotenv()
api_key = os.getenv('api_key')

In [93]:
url = 'https://api.elsevier.com/content/search/scopus' # URL da API do Scopus
headers = { # Cabeçalhos da requisição
    'Accept': 'application/json', # Formato de resposta esperado
    'X-ELS-APIKey': api_key # Chave da API para autenticação (ela deve ser definida no arquivo .env)
}

params = { # Parâmetros da requisição
    'query': 'TITLE-ABS-KEY("machine learning" AND "marketing" AND "consumer behavior") AND DOCTYPE(ar)', # Consulta de pesquisa
    'count': 25, # Número de resultados por página
    'start': 0 # Índice inicial dos resultados (0 para começar do primeiro resultado)
}

response = requests.get(url, headers=headers, params=params) # Fazendo a requisição GET para a API do Scopus

print(response.status_code) # Exibindo o código de status da resposta HTTP

200


In [94]:
# Recuperando todos os resultados
total_results = int(response.json()['search-results']['opensearch:totalResults'])
# Exibindo o total de resultados
print(f'Total de resultados: {total_results}')
# Verificando o número de resultados retornados
results_returned = response.json()['search-results']['opensearch:itemsPerPage']
print(f'Resultados retornados: {results_returned}')

Total de resultados: 141
Resultados retornados: 25


In [95]:
# Lista para armazenar os resultados
all_results = []

# Criando loop para recuperar todos os resultados
for start in range(0, total_results, params['count']):              # Itera sobre os resultados, incrementando o índice inicial
    params['start'] = start                                         # Atualiza o índice inicial na consulta
    response = requests.get(url, headers=headers, params=params)    # Faz a requisição GET com os parâmetros atualizados
    if response.status_code == 200:                                 # Verifica se a requisição foi bem-sucedida 
        data = response.json()                                      # Converte a resposta JSON em um dicionário
        entries = data.get('search-results', {}).get('entry', [])   # Extrai os resultados da pesquisa
        all_results.extend(entries)                                 # acumula os resultados 
        print(f'Resultados de {start} a {start + params["count"]}') # Exibe o intervalo de resultados recuperados
    else:                                                           # Se a requisição falhar, exibe o código de status
        print(f'Erro ao recuperar dados: {response.status_code}')   # Se houver erro, exibe o código de status
        break                                                       # Interrompe o loop se houver erro na requisição

# Salvando todos os resultados no CSV/Excel
df = pd.DataFrame(all_results)                      # Cria um DataFrame a partir da lista de resultados
df.to_csv('scopus_results.csv', index=False)        # Salva o DataFrame em um arquivo CSV
df.to_excel('scopus_results.xlsx', index=False)     # Salva o DataFrame em um arquivo Excel

Resultados de 0 a 25
Resultados de 25 a 50
Resultados de 50 a 75
Resultados de 75 a 100
Resultados de 100 a 125
Resultados de 125 a 150


PermissionError: [Errno 13] Permission denied: 'scopus_results.xlsx'